In [50]:
my_env = !which python  # MUST RUN ON NLP env (for pytorch)
print(my_env[0])
assert '/nlp/' in my_env[0], 'Wrong conda env!'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/mcirtain/miniconda3/envs/nlp/bin/python


In [51]:
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
import json
%matplotlib inline
from urllib.request import urlopen
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si
import requests

In [52]:
# url_chris = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/%23chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.json'
# #url= 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/project_data.2023-03-11_13.11.55.783862.json'
# df = pd.read_json(url_chris)
# df.info()


In [53]:
# nltk.download('vader_lexicon')

In [54]:
# df.head()

In [55]:
# sia = si()
# df['polarity_score'] = df.text.apply(lambda x: sia.polarity_scores(x))

In [56]:
# df_polar = df[['text','polarity_score']].copy()
# df_polar.head()

In [57]:
# print(df_polar.values[:1])

In [58]:
# df1 = pd.DataFrame(df, columns = ["text", "public_metrics.retweet_count", "public_metrics.reply_count",	"public_metrics.like_count", "public_metrics.quote_count", "public_metrics.impression_count", 'polarity_score'])

# df1 = df1.rename(columns= {"public_metrics.retweet_count":'retweet', "public_metrics.reply_count":"reply",	"public_metrics.like_count":"like", "public_metrics.quote_count":'quote', "public_metrics.impression_count":"impression"})
# df1


In [59]:
# df_like = df1.sort_values("like", ascending = False)
# df_like = df_like.head(300)
# df_like

In [60]:
# df_retweet = df1.sort_values("retweet", ascending = False)
# df_retweet = df_retweet.head(300)
# df_retweet

In [61]:
# verification
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.0396, 0.3243, 0.4825],
        [0.2434, 0.1244, 0.8184],
        [0.5320, 0.4462, 0.9252],
        [0.6833, 0.1960, 0.6324],
        [0.4310, 0.2436, 0.1119]])


In [62]:
import torch.nn as nn
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
nltk.download('stopwords')
import gzip
import gensim 
import logging
logging.basicConfig(format= '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import multiprocessing #import process
from gensim.models.word2vec import Word2Vec
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mcirtain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load data

In [63]:
url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/normalized_annotated_all_2023-04-07_12.43.49.255973.tsv'

df = pd.read_csv(url, sep='\t')
df.shape

(30611, 37)

In [64]:
# Try to fix the following block...
from bs4 import BeautifulSoup

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['clean_text'] = df['text'].apply(clean_text)

In [65]:
# REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
# BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))

# def clean_text(text):
#     """
#         text: a string
        
#         return: modified initial string
#     """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
#     text = text.lower() # lowercase text
#     text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
#     text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
#     #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
#     return text
    
# df['text'] = df['text'].apply(clean_text)


### Need Help I guess

In [66]:
# # Try to fix next cell:
# from transformers import AutoConfig, AutoModel, AutoTokenizer

# """
# Repo id must use alphanumeric chars 
# or '-', '_', '.', 
# '--' and '..' are forbidden, 
# '-' and '.' cannot start or end the name, 
# max length is 96: '/twitter-roberta-base-sentiment'.
# """
# MODEL = "cardiffnlp/twitter-roberta-base-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(MODEL)



In [67]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
# #MODEL = f"/twitter-roberta-base-{task}"  # mc change
# MODEL = "cardiffnlp/twitter-roberta-base-sentiment"


# tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)


In [68]:
# text = "Good night 😊"
# tweet = preprocess(text)
# print(f'preprocessed: {tweet}')
# encoded_input = tokenizer(text, return_tensors='pt')
# print(f'Encoded input: {encoded_input}')
# output = model(**encoded_input)
# print(f'output: {output}')
# scores = output[0][0].detach().numpy()
# print(f'scores: {scores}')
# scores = softmax(scores)
# print(f'softmax scores: {scores}')
# rankiing = np.argsort(scores)
# print(f'ranking: {ranking}')
# ranking = ranking[:: -1]
# print(f'ranking: {ranking}')
# for i in range(scores.shape[0]):
#     print(f'i: {i}')
#     l = labels[ranking[i]]
#     print(f'label l: {l}')
#     s = scores[ranking[i]]
#     print(f'score s: {s}')
    
# print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")   ##### WARNING - Not consistent

# row = {'text' : tweet, l: [np.round(float(s),4)]}  ##### WARNING - Not consistent
# print(f'row: {row}')


In [69]:
# text = "Good night 😊"
# tweet = preprocess(text)
# print(f'preprocessed: {tweet}')
# encoded_input = tokenizer(text, return_tensors='pt')
# print(f'Encoded input: {encoded_input}')
# output = model(**encoded_input)
# print(f'output: {output}')
# scores = output[0][0].detach().numpy()
# print(f'scores: {scores}')
# scores = softmax(scores)
# print(f'softmax scores: {scores}')
# print(f'Max score and location:', scores.max(), scores.argmax())
# sentiments = ['negative', 'neutral', 'positive']
# print(f'\nRoBERTa Prediction: {sentiments[scores.argmax()]}, Probability: {scores.max()}')
# rankiing = np.argsort(scores)
# print(f'ranking: {ranking}')
# ranking = ranking[:: -1]
# print(f'ranking: {ranking}')
# for i in range(scores.shape[0]):
#     print(f'i: {i}')
#     l = labels[ranking[i]]
#     print(f'label l: {l}')
#     s = scores[ranking[i]]
#     print(f'score s: {s}')
    
# # print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")   ##### WARNING - Not consistent

# # row = {'text' : tweet, l: [np.round(float(s),4)]}  ##### WARNING - Not consistent
# # print(f'row: {row}')
# row = {'text': tweet, sentiments[scores.argmax()]: [np.round(scores.max(),4)]}
# print(f'row: {row}')


In [70]:
#torch.nn - following a blog I found for roberta...
# https://medium.com/@amanabdulla296/sentiment-analysis-with-vader-and-twitter-roberta-2ede7fb78909


# twitter-roberta-base-sentiment model works better with minimalistic amount of manipulation to the data
# just let the model know that there is username http links
# def preprocess_new(text):
#     new_text = []
#     for t in text.split(" "):
#         t = "@user" if t.startswith("@") and len(t) > 1 else t
#         t = "http" if t.startswith("http") else t
#         new_text.append(t)
#     return " ".join(new_text)

import torch.nn as nn

# TEMPORARY: SHRINK DATA FOR FASTER EVAL
# df = df.iloc[:10]
# print(df.shape)
    
# apply the preprocessing functions to the content
df["roberta_content"] = df["text"].apply(preprocess)
# model, tokenizer and so on are located in the following repo
model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
roberta_model = AutoModelForSequenceClassification.from_pretrained(model_path)
# a function that takes text and model to calculate probability of each sentiment
def sentiment_analyzer(text, model):
    encoded_input = tokenizer(text, return_tensors="pt")
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = np.round(softmax(scores), 2)
    scores_dict = {"neg": scores[0], "neu": scores[1], "pos": scores[2]}
    return scores_dict

# apply the roberta function
df["roberta_probabilities"] = df["roberta_content"].apply(sentiment_analyzer, model=roberta_model)

# since roberta model returned probability of each sentiment as a dictionary
# let's convert each probaility into separate column
probabilities = df["roberta_probabilities"].apply(pd.Series)
df = df.join(probabilities, lsuffix='left', rsuffix='right')
df = df.drop("roberta_probabilities", axis=1)

# now calculate the polarity for each text by:
# first multiplying each probability to its weights (-1=> negative, 0=>neutral and +1=>positive)
# then sum the values and pass through Tanh function to scale values from -1 up to +1
# finally we can assign labels for each text, depending on the polarity, e.g. -1.0 until -0.25 negavite
polarity_weights = torch.tensor([-1, 0, 1])
print(f"df columns: {[x for x in df.columns if 'neu' in x or 'neg' in x or 'pos' in x]}")
probs = torch.tensor(df[["neg", "neu", "pos"]].values)
polarity = polarity_weights * probs
polarity = polarity.sum(dim=-1)
polarity_scaled = nn.Tanh()(polarity)
df["roberta_polarity"] = polarity_scaled.numpy()
df["roberta_sentiment"] = pd.cut(df["roberta_polarity"],
    bins=[-1.0, -0.25, 0.25, 1.0],labels=["negative", "neutral", "positive"],)
df = df.drop(["neu", "neg", "pos"], axis=1)
df.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


df columns: ['neg', 'neu', 'pos']


,Unnamed: 0.1,Unnamed: 0,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,normalized_textblob_Subjectivity,normalized_vader_Polarity,normalized_vader_Subjectivity,normalized_AFINN_scores,normalized_SentiWordNet_scores,our_label,annotator,roberta_content,roberta_polarity,roberta_sentiment
0,0,0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry_bucky hes lucky a #chrisrocklive ...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",...,0.527778,0.176602,0.640625,0.448980,0.444444,NaN,NaN,RT @user He’s lucky a #ChrisRockLive was the w...,-0.635149,negative
1,1,1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt 1_ofakindnocap chris rock we all been chea...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.000000,0.217017,0.529018,0.367347,0.333333,NaN,NaN,RT @user Chris Rock: “we all been cheated on.....,-0.635149,negative
2,2,2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt 1_ofakindnocap chris rock we all been chea...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",...,0.000000,0.217017,0.529018,0.367347,0.333333,NaN,NaN,RT @user Chris Rock: “we all been cheated on.....,-0.635149,negative
3,3,3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working out and watching the...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",...,0.535714,0.702710,0.191964,0.489796,0.412698,NaN,NaN,RT @user Working out and watching the @user @u...,0.388473,positive
4,4,4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done ill have a few ...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",...,0.535714,0.702710,0.219866,0.489796,0.412698,NaN,NaN,RT @user Workout done. I’ll have a few comment...,0.168381,neutral


In [71]:
# # Can I announce when we're done?
# import os
# os.system("say 'hey google'")
# os.system("say 'broadcast'")
# os.system("say 'done running your roberta notebook'")

In [72]:
# import os
# os.system("say 'hey google'")
# os.system("say 'broadcast'")
# os.system("say 'done running your roberta notebook'")

In [73]:
# %%time

# # reimplementation...
# text = "Good night 😊"
# #sentiment_df = pd.DataFrame(columns = ['text'])
# sentiments = ['negative', 'neutral', 'positive']

# df['roberta_raw_negative'] = None
# df['roberta_raw_neutral'] = None
# df['roberta_raw_positive'] = None
# df['roberta_label'] = None
# df['roberta__neg_softmax_score'] = None
# df['roberta__neut_softmax_score'] = None
# df['roberta__pos_softmax_score'] = None
# #for tweet in df['text']:
# for idx, row in df.iterrows():
#     # classify each tweet, row by row
#     encoded_input = tokenizer(row['text'], return_tensors='pt')
#     output = model(**encoded_input)
#     scores = output[0][0].detach().numpy()
    
#     scores = softmax(scores)
    
#     # update our data
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_label'] = sentiments[scores.argmax()]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_argmax'] = scores.max()
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_negative'] = scores[0]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_neutral'] = scores[1]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_positive'] = scores[2]
    
#     if idx % 500 == 0:
#         print(f'Through {idx} rows')
    
# df.head()


In [74]:
# text = "Good night 😊"
# sentiment_df = pd.DataFrame(columns = ['text'])

# df['roberta_label'] = None
# df['roberta_score'] = None
# for tweet in df['text']:
#   tweet = preprocess(tweet)

#   encoded_input = tokenizer(text, return_tensors='pt')
#   output = model(**encoded_input)
#   scores = output[0][0].detach().numpy()
#   scores = softmax(scores)



#   ranking = np.argsort(scores)
#   ranking = ranking[::-1]
#   for i in range(scores.shape[0]):
#     l = labels[ranking[i]]
#     s = scores[ranking[i]]
#     print(f"tweet : {tweet} {i+1}) {l} {np.round(float(s), 4)}")
#     row = {'text' : tweet, l: [np.round(float(s),4)]}
#     #Append row to the sentiment data frame
#     sentiment_df = sentiment_df.append(row, ignore_index = True)  



In [75]:
# Can we vectorize this?


In [76]:
df.tail(3)

,Unnamed: 0.1,Unnamed: 0,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,normalized_textblob_Subjectivity,normalized_vader_Polarity,normalized_vader_Subjectivity,normalized_AFINN_scores,normalized_SentiWordNet_scores,our_label,annotator,roberta_content,roberta_polarity,roberta_sentiment
30608,30608,32288,1633873447897149440,en,everyone,2023-03-09 16:52:40+00:00,rt kajakallas 79 years ago soviet planes bomb...,RT @kajakallas: 79 years ago Soviet planes bom...,1250015178248290304,"[{'type': 'retweeted', 'id': '1633741101885542...",...,0.0,0.111724,0.357143,0.367347,0.269841,NaN,NaN,RT @user 79 years ago Soviet planes bombed Tal...,-0.691069,negative
30609,30609,32289,1633873447108632576,en,everyone,2023-03-09 16:52:40+00:00,rt sternenko today 4 civilians were killed by...,RT @sternenko: Today 4 civilians were killed b...,1495109228876468224,"[{'type': 'retweeted', 'id': '1633803881447604...",...,0.0,0.276121,0.393973,0.448980,0.428571,NaN,NaN,RT @user Today 4 civilians were killed by russ...,-0.701374,negative
30610,30610,32290,1633873445154091008,en,everyone,2023-03-09 16:52:39+00:00,rt sunshinelove68 how did the us lose six of ...,RT @Sunshinelove68: 💥How Did the U.S. Lose Six...,980084548376584192,"[{'type': 'retweeted', 'id': '1633861161144958...",...,0.0,0.104989,0.717634,0.367347,0.476190,NaN,NaN,RT @user 💥How Did the U.S. Lose Six Of its Nuc...,-0.735222,negative


In [77]:
# df.to_csv('../../data/roberta_on_chris.tsv', sep='\t')

In [78]:
# # Try to run on all of our data:
# url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/normalized_annotated_all_2023-04-07_12.43.49.255973.tsv'

# df = pd.read_csv(url, sep='\t')
# df.shape

In [79]:
# %%time

# # reimplementation...
# #text = "Good night 😊"
# #sentiment_df = pd.DataFrame(columns = ['text'])
# sentiments = ['negative', 'neutral', 'positive']

# df['clean_text'] = df['text'].apply(clean_text)

# df['roberta_negative'] = None
# df['roberta_neutral'] = None
# df['roberta_positive'] = None
# df['roberta_label'] = None
# df['roberta_score'] = None

# for idx, row in df.iterrows():
#     # classify each tweet, row by row
#     encoded_input = tokenizer(row['clean_text'], return_tensors='pt')
#     output = model(**encoded_input)
#     scores = output[0][0].detach().numpy()
#     scores = softmax(scores)
    
#     # update our data
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_label'] = sentiments[scores.argmax()]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_argmax'] = scores.max()
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_negative'] = scores[0]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_neutral'] = scores[1]
#     df.loc[df['conversation_id'] == row['conversation_id'], 'roberta_positive'] = scores[2]
    
#     if idx % 500 == 0:
#         print(f'Through {idx} rows')
    
# df.head()

In [80]:
# df.to_csv('roberta_all.tsv', sep='\t')

In [81]:
# df.to_json('../../data/roberta_all.json')

In [82]:
df.to_json('../../data/roberta_polarity_scores_not_normalized_04162023.0743.json')